**Baseline Model Summary (Non–Deep Learning Approach)**

To create a simple classical machine-learning baseline using traditional computer vision features, before moving to deep learning models. This baseline provides a reference point to compare CNN, MobileNet, ResNet, and EfficientNet performance.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [ ]:
BASE = "/content/drive/MyDrive/CMPE 593/datasets/processed_datasets/NTHU-DDD"
TRAIN_DIR = os.path.join(BASE, "train")
VAL_DIR = os.path.join(BASE, "val")
TEST_DIR = os.path.join(BASE, "test")

print("Train:", TRAIN_DIR)
print("Val:", VAL_DIR)
print("Test:", TEST_DIR)


Train: /content/drive/MyDrive/CMPE 593/datasets/processed_datasets/NTHU-DDD/train
Val: /content/drive/MyDrive/CMPE 593/datasets/processed_datasets/NTHU-DDD/val
Test: /content/drive/MyDrive/CMPE 593/datasets/processed_datasets/NTHU-DDD/test


Dataset structure:

*   train/drowsy, train/notdrowsy
*   val/drowsy, val/notdrowsy
*   test/drowsy, test/notdrowsy

Only a subset of each split was used (≈1500 per class) to keep computation fast.

In [ ]:
MAX_PER_CLASS = 1500

Why subset?
- Classical ML feature extraction is slow for large datasets.
- Subset maintains representativeness but reduces computation by 10–20×.
- Perfectly acceptable for baselines (goal is comparison, not SOTA results).

**ORB Feature Extractor**

Instead of HOG (too slow), we used:

ORB (Oriented FAST and Rotated BRIEF)
- very fast
- rotation-invariant
- suitable for small images
- produces compact descriptors

ORB settings:

- nfeatures=300
- Resize images to 128×128

Always return a 300-dimensional feature vector (pad or truncate)

In [ ]:
IMG_SIZE = (128, 128)
orb = cv2.ORB_create(nfeatures=300)
FEATURE_DIM = 300

In [ ]:
def extract_orb_features(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, IMG_SIZE)

    keypoints, descriptors = orb.detectAndCompute(img, None)

    if descriptors is None:
        return np.zeros(FEATURE_DIM)

    flattened = descriptors.flatten()

    if flattened.size >= FEATURE_DIM:
        return flattened[:FEATURE_DIM]
    else:
        padded = np.zeros(FEATURE_DIM)
        padded[:flattened.size] = flattened
        return padded

Load Images from Folder + Extract ORB Features

In [ ]:
def load_split(split_dir):
    X, y = [], []

    for label_name in ["drowsy", "notdrowsy"]:
        class_dir = os.path.join(split_dir, label_name)
        label = 1 if label_name == "drowsy" else 0

        files = os.listdir(class_dir)
        np.random.shuffle(files)  # shuffle before selecting subset

        selected = files[:MAX_PER_CLASS]   # <-- SELECT ONLY SUBSET

        for filename in tqdm(selected, desc=f"Loading {label_name}"):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(class_dir, filename)
                features = extract_orb_features(path)
                X.append(features)
                y.append(label)

    return np.array(X), np.array(y)


Load Train / Val / Test Sets

In [ ]:
print("Loading TRAIN dataset...")
X_train, y_train = load_split(TRAIN_DIR)

print("\nLoading VAL dataset...")
X_val, y_val = load_split(VAL_DIR)

print("\nLoading TEST dataset...")
X_test, y_test = load_split(TEST_DIR)


Loading TRAIN dataset...


Loading notdrowsy: 100%|██████████| 1500/1500 [33:16<00:00,  1.33s/it]



Loading VAL dataset...


Loading notdrowsy: 100%|██████████| 1500/1500 [02:47<00:00,  8.93it/s] 



Loading TEST dataset...


Loading notdrowsy: 100%|██████████| 1500/1500 [02:41<00:00,  9.29it/s] 


Train Classical Machine Learning Baseline (Linear SVM)

Linear SVM (Linear Support Vector Classifier)
Used because:

- fast training
- works well with high-dimensional feature vectors
- widely used as a traditional CV baseline

No neural networks involved — completely classical ML.

In [ ]:
print("Training Linear SVM baseline...")
model = LinearSVC()
model.fit(X_train, y_train)


Training Linear SVM baseline...


LinearSVC()

In [ ]:
joblib.dump(model, os.path.join(BASE, "baseline_orb_svm.pkl"))

['/content/drive/MyDrive/CMPE 593/datasets/processed_datasets/NTHU-DDD/baseline_orb_svm.pkl']

Evaluate

In [ ]:
print(classification_report(y_val, model.predict(X_val)))
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.69      0.65      0.67      1500
           1       0.67      0.70      0.69      1500

    accuracy                           0.68      3000
   macro avg       0.68      0.68      0.68      3000
weighted avg       0.68      0.68      0.68      3000

              precision    recall  f1-score   support

           0       0.71      0.67      0.69      1500
           1       0.69      0.72      0.70      1500

    accuracy                           0.70      3000
   macro avg       0.70      0.70      0.70      3000
weighted avg       0.70      0.70      0.70      3000



Training Flow

1. Load subset of images from each class.
2. Extract ORB features for each image.
3. Build feature matrix X and labels y.
4. Train a Linear SVM classifier on X_train.
5. Evaluate on:
- Validation set
- Test set

Metrics used:
- Accuracy
- Precision
- Recall
- F1-score

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

print("Training Logistic Regression...")
logreg = LogisticRegression(max_iter=2000)
logreg.fit(X_train, y_train)

print("\n=== Logistic Regression (VAL) ===")
y_val_pred = logreg.predict(X_val)
print(classification_report(y_val, y_val_pred))

print("VAL Accuracy:", accuracy_score(y_val, y_val_pred))

print("\n=== Logistic Regression (TEST) ===")
y_test_pred = logreg.predict(X_test)
print(classification_report(y_test, y_test_pred))

print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))


Training Logistic Regression...

=== Logistic Regression (VAL) ===
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      1500
           1       0.67      0.69      0.68      1500

    accuracy                           0.68      3000
   macro avg       0.68      0.68      0.68      3000
weighted avg       0.68      0.68      0.68      3000

VAL Accuracy: 0.6766666666666666

=== Logistic Regression (TEST) ===
              precision    recall  f1-score   support

           0       0.70      0.67      0.69      1500
           1       0.69      0.72      0.70      1500

    accuracy                           0.70      3000
   macro avg       0.70      0.70      0.70      3000
weighted avg       0.70      0.70      0.70      3000

TEST Accuracy: 0.6953333333333334


Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

print("\n=== Random Forest (VAL) ===")
y_val_pred = rf.predict(X_val)
print(classification_report(y_val, y_val_pred))
print("VAL Accuracy:", accuracy_score(y_val, y_val_pred))

print("\n=== Random Forest (TEST) ===")
y_test_pred = rf.predict(X_test)
print(classification_report(y_test, y_test_pred))
print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))


Training Random Forest...

=== Random Forest (VAL) ===
              precision    recall  f1-score   support

           0       0.78      0.80      0.79      1500
           1       0.79      0.77      0.78      1500

    accuracy                           0.78      3000
   macro avg       0.78      0.78      0.78      3000
weighted avg       0.78      0.78      0.78      3000

VAL Accuracy: 0.784

=== Random Forest (TEST) ===
              precision    recall  f1-score   support

           0       0.78      0.79      0.79      1500
           1       0.79      0.78      0.78      1500

    accuracy                           0.79      3000
   macro avg       0.79      0.79      0.79      3000
weighted avg       0.79      0.79      0.79      3000

TEST Accuracy: 0.7856666666666666


k-Nearest Neighbors (kNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print("Training kNN (k=5)...")
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train, y_train)

print("\n=== kNN (VAL) ===")
y_val_pred = knn.predict(X_val)
print(classification_report(y_val, y_val_pred))
print("VAL Accuracy:", accuracy_score(y_val, y_val_pred))

print("\n=== kNN (TEST) ===")
y_test_pred = knn.predict(X_test)
print(classification_report(y_test, y_test_pred))
print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))


Training kNN (k=5)...

=== kNN (VAL) ===
              precision    recall  f1-score   support

           0       0.77      0.70      0.73      1500
           1       0.72      0.79      0.75      1500

    accuracy                           0.74      3000
   macro avg       0.74      0.74      0.74      3000
weighted avg       0.74      0.74      0.74      3000

VAL Accuracy: 0.742

=== kNN (TEST) ===
              precision    recall  f1-score   support

           0       0.78      0.68      0.73      1500
           1       0.72      0.81      0.76      1500

    accuracy                           0.75      3000
   macro avg       0.75      0.75      0.75      3000
weighted avg       0.75      0.75      0.75      3000

TEST Accuracy: 0.7463333333333333


Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

print("Training Gaussian Naive Bayes...")
nb = GaussianNB()
nb.fit(X_train, y_train)

print("\n=== Naive Bayes (VAL) ===")
y_val_pred = nb.predict(X_val)
print(classification_report(y_val, y_val_pred))
print("VAL Accuracy:", accuracy_score(y_val, y_val_pred))

print("\n=== Naive Bayes (TEST) ===")
y_test_pred = nb.predict(X_test)
print(classification_report(y_test, y_test_pred))
print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))


Training Gaussian Naive Bayes...

=== Naive Bayes (VAL) ===
              precision    recall  f1-score   support

           0       0.64      0.68      0.66      1500
           1       0.66      0.62      0.64      1500

    accuracy                           0.65      3000
   macro avg       0.65      0.65      0.65      3000
weighted avg       0.65      0.65      0.65      3000

VAL Accuracy: 0.6523333333333333

=== Naive Bayes (TEST) ===
              precision    recall  f1-score   support

           0       0.66      0.69      0.67      1500
           1       0.67      0.64      0.66      1500

    accuracy                           0.66      3000
   macro avg       0.67      0.66      0.66      3000
weighted avg       0.67      0.66      0.66      3000

TEST Accuracy: 0.6646666666666666


XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
print("Training XGBoost...")
xgb = XGBClassifier(
    max_depth=6,
    n_estimators=300,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss'
)

xgb.fit(X_train, y_train)

print("\n=== XGBoost (VAL) ===")
y_val_pred = xgb.predict(X_val)
print(classification_report(y_val, y_val_pred))
print("VAL Accuracy:", accuracy_score(y_val, y_val_pred))

print("\n=== XGBoost (TEST) ===")
y_test_pred = xgb.predict(X_test)
print(classification_report(y_test, y_test_pred))
print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))

Training XGBoost...

=== XGBoost (VAL) ===
              precision    recall  f1-score   support

           0       0.79      0.79      0.79      1500
           1       0.79      0.79      0.79      1500

    accuracy                           0.79      3000
   macro avg       0.79      0.79      0.79      3000
weighted avg       0.79      0.79      0.79      3000

VAL Accuracy: 0.792

=== XGBoost (TEST) ===
              precision    recall  f1-score   support

           0       0.82      0.79      0.80      1500
           1       0.80      0.83      0.81      1500

    accuracy                           0.81      3000
   macro avg       0.81      0.81      0.81      3000
weighted avg       0.81      0.81      0.81      3000

TEST Accuracy: 0.8076666666666666
